In [2]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
import time

# Set up the Chrome WebDriver
service = Service('path_to_chromedriver')  # Replace with the path to your ChromeDriver
driver = webdriver.Chrome(service=service)

# Open a TikTok video URL
video_url = 'https://www.tiktok.com/@anuu__21/photo/7415536407583755525'  # Replace with the TikTok video URL
driver.get(video_url)

# Allow the page to load
time.sleep(5)

# Scroll down to load comments
actions = ActionChains(driver)
for _ in range(5):  # Adjust to load more comments
    actions.move_to_element(driver.find_element(By.TAG_NAME, 'body')).perform()
    time.sleep(2)

# Extract comments
comments = []
comment_elements = driver.find_elements(By.XPATH, '//div[@data-e2e="comment-list"]//p[@data-e2e="comment-level-text"]')

for comment in comment_elements:
    comments.append(comment.text)

# Save comments to a text file
with open("comments.txt", "w", encoding="utf-8") as txt_file:
    for comment in comments:
        txt_file.write(comment + "\n")

# Save comments to a CSV file
with open("comments.csv", "w", encoding="utf-8", newline="") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["Comment"])  # Add a header
    for comment in comments:
        writer.writerow([comment])

# Print a success message
print(f"Saved {len(comments)} comments to 'comments.txt' and 'comments.csv'")

# Close the driver
driver.quit()


NoSuchDriverException: Message: Unable to obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location


In [15]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.options import Options
import time

# Set up Edge WebDriver with User-Agent
options = Options()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
service = Service('C:/Users/pc/Downloads/edgedriver_win64/msedgedriver.exe')  # Replace with the correct path
driver = webdriver.Edge(service=service, options=options)

# Open TikTok video
video_url = 'https://www.tiktok.com/@anuu__21/photo/7415536407583755525'  # Replace with the video URL
driver.get(video_url)
time.sleep(10)  # Allow the page to load fully

# Scroll and collect comments
comments = []
try:
    while len(comments) < 20:  # Stop after collecting 20 comments
        # Scroll to load more comments
        driver.execute_script("window.scrollBy(10, 1000);")
        time.sleep(2)  # Wait for comments to load

        # Extract comments using XPath
        comment_elements = driver.find_elements(By.XPATH, '//div[@data-e2e="comment-list"]//p[@data-e2e="comment-level-text"]')

        # Add unique comments to the list
        for element in comment_elements:
            comment_text = element.text.strip()
            if comment_text not in comments:  # Avoid duplicates
                comments.append(comment_text)

        print(f"Collected {len(comments)} comments so far...")  # Debugging output

except Exception as e:
    print(f"Error extracting comments: {e}")

# Save comments to a text file
if comments:
    with open("comments.txt", "w", encoding="utf-8") as txt_file:
        for comment in comments[:20]:  # Save only the first 20 comments
            txt_file.write(comment + "\n")
    print(f"Saved {len(comments[:20])} comments to 'comments.txt'")
else:
    print("No comments to save.")

# Close the driver
driver.quit()


Collected 0 comments so far...
Collected 0 comments so far...
Collected 0 comments so far...
Collected 0 comments so far...
Collected 0 comments so far...
Collected 0 comments so far...
Collected 0 comments so far...
Collected 0 comments so far...
Collected 0 comments so far...
Collected 0 comments so far...
Collected 0 comments so far...
Collected 0 comments so far...
Collected 0 comments so far...
Collected 0 comments so far...
Collected 0 comments so far...
Collected 0 comments so far...
Collected 0 comments so far...
Collected 0 comments so far...
Collected 0 comments so far...
Collected 0 comments so far...
Collected 0 comments so far...
Collected 0 comments so far...
Collected 0 comments so far...
Collected 0 comments so far...
Collected 0 comments so far...
Collected 0 comments so far...
Collected 0 comments so far...
Collected 0 comments so far...
Collected 0 comments so far...
Collected 0 comments so far...
Collected 0 comments so far...
Collected 0 comments so far...
Collecte

In [44]:
# !pip install playwright
# !playwright install
!pip install nest-asyncio


In [46]:
import asyncio
from playwright.async_api import async_playwright
import nest_asyncio

# Apply the nest_asyncio patch
nest_asyncio.apply()

async def scrape_tiktok_comments():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        context = await browser.new_context(
            user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        )
        page = await context.new_page()

        video_url = 'https://www.tiktok.com/@anuu__21/photo/7415536407583755525'
        await page.goto(video_url)
        await page.wait_for_timeout(10000)  # Wait for comments to load

        comments = []
        for _ in range(10):  # Adjust number of scrolls
            await page.mouse.wheel(0, 1000)  # Scroll down
            await page.wait_for_timeout(2000)

            comment_elements = await page.locator('//p[@data-e2e="comment-level-text"]').all()
            for element in comment_elements:
                comment_text = await element.inner_text()
                if comment_text not in comments:
                    comments.append(comment_text)

            if len(comments) >= 20:
                break

        if comments:
            with open("comments.txt", "w", encoding="utf-8") as f:
                for comment in comments[:20]:
                    f.write(comment + "\n")
            print(f"Saved {len(comments[:20])} comments to 'comments.txt'")
        else:
            print("No comments found.")

        await browser.close()

await scrape_tiktok_comments()


NotImplementedError: 

In [7]:
import requests
from requests.exceptions import ConnectionError, Timeout

def fetch_and_save_url(url, retries=3, timeout=10, output_file='output.xlsx'):
    for attempt in range(retries):
        try:
            response = requests.get(url, timeout=timeout)
            response.raise_for_status()  # Raise an error for bad responses (4xx or 5xx)
            
            # Save the response content to a file
            with open(output_file, 'w', encoding='utf-8') as file:
                file.write(response.text)
            
            print(f"Content successfully fetched and saved to '{output_file}'.")
            return response.text  # Return the content of the response
        except ConnectionError as e:
            print(f"Connection error: {e}. Attempt {attempt + 1} of {retries}.")
        except Timeout as e:
            print(f"Request timed out: {e}. Attempt {attempt + 1} of {retries}.")
        except Exception as e:
            print(f"An error occurred: {e}. Attempt {attempt + 1} of {retries}.")
    
    print("All attempts failed.")
    return None

# Example usage
url = 'https://www.tiktok.com/@anuu__21/photo/7415536407583755525'  # Replace with your target URL
output_file = 'comments.xlsx'  # Specify the output file name
content = fetch_and_save_url(url, output_file=output_file)
if content:
    print("Successfully fetched and saved the content.")
else:
    print("Failed to fetch the content.")


Connection error: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)). Attempt 1 of 3.
Content successfully fetched and saved to 'comments.xlsx'.
Successfully fetched and saved the content.


In [17]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
import pandas as pd
import time

def fetch_tiktok_content_with_selenium(url, output_file='tiktok_content.csv'):
    # Path to Edge WebDriver (update this path)
    driver_path = "C:/Users/pc/Downloads/edgedriver_win64/msedgedriver.exe"  # Replace with your actual Edge WebDriver path

    # Set up Edge options
    edge_options = Options()
    edge_options.add_argument("--headless")  # Run in headless mode (no GUI)
    edge_options.add_argument("--disable-gpu")
    edge_options.add_argument("--no-sandbox")

    try:
        # Initialize WebDriver for Edge
        driver = webdriver.Edge(service=Service(driver_path), options=edge_options)

        # Open the TikTok URL
        driver.get(url)
        time.sleep(5)  # Allow content to load

        # Get the page source
        page_source = driver.page_source

        # Save content to CSV
        df = pd.DataFrame([{"content": page_source}])
        df.to_csv(output_file, index=False)

        print(f"Content saved to {output_file}")
        return True
    except Exception as e:
        print(f"Error: {e}")
        return False
    finally:
        if 'driver' in locals():
            driver.quit()

# Example usage
fetch_tiktok_content_with_selenium("https://www.tiktok.com/@anuu__21/photo/7415536407583755525")


Content saved to tiktok_content.csv


True

In [19]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
import pandas as pd
import time

def fetch_tiktok_comments(url, output_file='tiktok_comments.csv'):
    # Path to Edge WebDriver (update this path)
    driver_path = "C:/Users/pc/Downloads/edgedriver_win64/msedgedriver.exe"  # Replace with your actual Edge WebDriver path

    # Set up Edge options
    edge_options = Options()
    edge_options.add_argument("--headless")  # Run in headless mode (no GUI)
    edge_options.add_argument("--disable-gpu")
    edge_options.add_argument("--no-sandbox")

    try:
        # Initialize WebDriver for Edge
        driver = webdriver.Edge(service=Service(driver_path), options=edge_options)

        # Open the TikTok URL
        driver.get(url)
        time.sleep(5)  # Allow content to load

        # Extract the user name (can be found by inspecting the profile name element)
        try:
            user_name = driver.find_element(By.CSS_SELECTOR, 'h1.share-title').text
        except Exception as e:
            user_name = 'Not found'

        # Extract the comments (you may need to adjust the selector based on the page structure)
        comments = []
        try:
            comment_elements = driver.find_elements(By.CSS_SELECTOR, 'div.comment-item p')
            for comment in comment_elements:
                comments.append(comment.text)
        except Exception as e:
            comments.append('No comments')

        # Prepare data for CSV (with user name and comments)
        data = {
            "User Name": [user_name],
            "Comments": [", ".join(comments)]  # Join all comments into a single string
        }
        
        # Save to CSV
        df = pd.DataFrame(data)
        df.to_csv(output_file, index=False)

        print(f"Content saved to {output_file}")
        return True
    except Exception as e:
        print(f"Error: {e}")
        return False
    finally:
        if 'driver' in locals():
            driver.quit()

# Example usage
fetch_tiktok_comments("https://www.tiktok.com/@anuu__21/photo/7415536407583755525")

Content saved to tiktok_comments.csv


True

In [25]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
import pandas as pd
import time

def fetch_tiktok_comments(url, output_file='tiktok_comments.csv'):
    # Path to Edge WebDriver (update this path)
    driver_path = "C:/Users/pc/Downloads/edgedriver_win64/msedgedriver.exe"  # Replace with your actual Edge WebDriver path

    # Set up Edge options
    edge_options = Options()
    edge_options.add_argument("--headless")  # Run in headless mode (no GUI)
    edge_options.add_argument("--disable-gpu")
    edge_options.add_argument("--no-sandbox")

    try:
        # Initialize WebDriver for Edge
        driver = webdriver.Edge(service=Service(driver_path), options=edge_options)

        # Open the TikTok URL
        driver.get(url)
        time.sleep(5)  # Allow content to load

        # Extract the user name (can be found by inspecting the profile name element)
        try:
            user_name = driver.find_element(By.CSS_SELECTOR, 'h1.share-title').text
        except Exception as e:
            user_name = 'Not found'

        # Extract the comments (you may need to adjust the selector based on the page structure)
        comments = []
        try:
            comment_elements = driver.find_elements(By.CSS_SELECTOR, 'div.comment-item p')
            for comment in comment_elements:
                comments.append(comment.text)
        except Exception as e:
            comments.append('No comments')

        # Prepare data for CSV (with user name and comments)
        data = {
            "User Name": [user_name],
            "Comments": [", ".join(comments)]  # Join all comments into a single string
        }

        # Save to CSV
        df = pd.DataFrame(data)
        df.to_csv(output_file, index=False)

        print(f"Content saved to {output_file}")
        return True
    except Exception as e:
        print(f"Error: {e}")
        return False
    finally:
        if 'driver' in locals():
            driver.quit()

# Example usage
url = "https://www.tiktok.com/@anuu__21/video/7415536407583755525"  # Ensure this is a video URL
output_file = 'tiktok_comments.csv'
fetch_tiktok_comments(url, output_file)

Content saved to tiktok_comments.csv


True

In [27]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
import pandas as pd
import time

def fetch_tiktok_comments_with_js(url, output_file='tiktok_comments.csv'):
    # Path to Edge WebDriver (update this path)
    driver_path = "C:/Users/pc/Downloads/edgedriver_win64/msedgedriver.exe"  # Replace with your actual Edge WebDriver path

    # Set up Edge options
    edge_options = Options()
    edge_options.add_argument("--headless")  # Run in headless mode (no GUI)
    edge_options.add_argument("--disable-gpu")
    edge_options.add_argument("--no-sandbox")

    try:
        # Initialize WebDriver for Edge
        driver = webdriver.Edge(service=Service(driver_path), options=edge_options)

        # Open the TikTok URL
        driver.get(url)
        time.sleep(5)  # Allow content to load

        # Use JavaScript to fetch user name
        try:
            user_name = driver.execute_script("""
                return document.querySelector('h1.share-title')?.innerText || 'Not found';
            """)
        except Exception as e:
            user_name = 'Not found'

        # Use JavaScript to fetch comments
        try:
            comments = driver.execute_script("""
                const commentElements = document.querySelectorAll('div.comment-item p');
                return Array.from(commentElements).map(comment => comment.innerText);
            """)
        except Exception as e:
            comments = []

        # Prepare data for CSV
        data = {
            "User Name": [user_name],
            "Comments": [", ".join(comments) if comments else "No comments"]
        }

        # Save to CSV
        df = pd.DataFrame(data)
        df.to_csv(output_file, index=False)

        print(f"Content saved to {output_file}")
        return True
    except Exception as e:
        print(f"Error: {e}")
        return False
    finally:
        if 'driver' in locals():
            driver.quit()

# Example usage
url = "https://www.tiktok.com/@anuu__21/photo/7415536407583755525"
output_file = 'tiktok_comments.csv'
fetch_tiktok_comments_with_js(url, output_file)


Content saved to tiktok_comments.csv


True

In [50]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
import pandas as pd
import time

def fetch_tiktok_comments_with_js(url, output_file='tiktok_comments.csv'):
    # Path to Edge WebDriver (update this path)
    driver_path = "C:/Users/pc/Downloads/edgedriver_win64/msedgedriver.exe"

    # Set up Edge options
    edge_options = Options()
    # Temporarily disable headless mode for debugging
    # Comment out the line below for debugging purposes
    # edge_options.add_argument("--headless")
    edge_options.add_argument("--disable-gpu")
    edge_options.add_argument("--no-sandbox")

    try:
        # Initialize WebDriver for Edge
        driver = webdriver.Edge(service=Service(driver_path), options=edge_options)

        # Open the TikTok URL
        driver.get(url)
        time.sleep(5)  # Allow some time for the page to load fully

        # Use JavaScript to extract the username
        try:
            user_name = driver.execute_script(
                """
                const usernameElement = document.querySelector('[data-e2e="user-title"]');
                return usernameElement ? usernameElement.innerText : 'Not found';
                """
            )
        except Exception as e:
            user_name = 'Not found'
            print(f"User name not found using JavaScript: {e}")

        # Use JavaScript to extract the comments
        try:
            comments = driver.execute_script(
                """
                const commentElements = document.querySelectorAll('[data-e2e="comment-item"] [data-e2e="comment-text"]');
                return Array.from(commentElements).map(el => el.innerText);
                """
            )
        except Exception as e:
            comments = []
            print(f"Comments not found using JavaScript: {e}")

        # Prepare data for CSV
        data = {
            "User Name": [user_name],
            "Comments": [", ".join(comments) if comments else "No comments"]
        }

        # Save to CSV
        df = pd.DataFrame(data)
        df.to_csv(output_file, index=False)

        print(f"Content saved to {output_file}")
        return True
    except Exception as e:
        print(f"Error: {e}")
        return False
    finally:
        if 'driver' in locals():
            driver.quit()

# Example usage
url = "https://www.tiktok.com/@anuu__21/photo/7415536407583755525"
output_file = 'tiktok_comments_with_js.csv'
fetch_tiktok_comments_with_js(url, output_file)


Content saved to tiktok_comments_with_js.csv


True

In [52]:
from TikTokApi import TikTokApi

# Initialize TikTok API
api = TikTokApi()

# Fetch a video
video = api.video(id="1234567890123456789")

# Get comments
for comment in video.comments:
    print(f"Comment: {comment.text}, Likes: {comment.likes_count}")

    # Fetch replies
    async for reply in comment.replies():
        print(f"    Reply: {reply.text}")


TypeError: 'method' object is not iterable

In [56]:
import pandas as pd
from TikTokApi import TikTokApi

def fetch_and_save_comments(video_id, output_file="comments.csv"):
    # Initialize TikTok API
    api = TikTokApi()

    try:
        # Fetch the video by its ID
        video = api.video(id=video_id)

        # Prepare a list to store comments
        comments_data = []

        # Fetch comments
        for comment in video.comments:
            comment_data = {
                "Comment ID": comment.id,
                "Author": comment.author.username,
                "Text": comment.text,
                "Likes": comment.likes_count
            }
            comments_data.append(comment_data)

            # Fetch replies for each comment (if any)
            async for reply in comment.replies():
                reply_data = {
                    "Comment ID": reply.id,
                    "Author": reply.author.username,
                    "Text": reply.text,
                    "Likes": reply.likes_count
                }
                comments_data.append(reply_data)

        # Convert the data to a DataFrame
        df = pd.DataFrame(comments_data)

        # Save to a CSV file
        df.to_csv(output_file, index=False)
        print(f"Comments have been saved to {output_file}")

    except Exception as e:
        print(f"An error occurred: {e}")

# Example Usage
video_id = "1234567890123456789"  # Replace with your TikTok video ID
output_file = "tiktok_comments.csv"
fetch_and_save_comments(video_id, output_file)


SyntaxError: 'async for' outside async function (424417065.py, line 26)

In [27]:
import pandas as pd
import asyncio
from TikTokApi import TikTokApi

async def fetch_and_save_comments(video_id, output_file="comments.csv"):
    # Initialize TikTok API
    api = TikTokApi()

    try:
        # Fetch the video by its ID
        video = api.video(id=video_id)

        # Prepare a list to store comments
        comments_data = []

        # Fetch comments
        for comment in video.comments:
            comment_data = {
                "Comment ID": comment.id,
                "Author": comment.author.username,
                "Text": comment.text,
                "Likes": comment.likes_count
            }
            comments_data.append(comment_data)

            # Fetch replies for each comment (if any)
            async for reply in comment.replies():
                reply_data = {
                    "Comment ID": reply.id,
                    "Author": reply.author.username,
                    "Text": reply.text,
                    "Likes": reply.likes_count
                }
                comments_data.append(reply_data)

        # Convert the data to a DataFrame
        df = pd.DataFrame(comments_data)

        # Save to a CSV file
        df.to_csv(output_file, index=False)
        print(f"Comments have been saved to {output_file}")

    except Exception as e:
        print(f"An error occurred: {e}")

# Run the async function using asyncio
video_id = "1234567890123456789"  # Replace with your TikTok video ID
output_file = "tiktok_comments.csv"

# Run the asynchronous fetch function
asyncio.run(fetch_and_save_comments(video_id, output_file))


RuntimeError: asyncio.run() cannot be called from a running event loop

In [25]:
import pandas as pd
import asyncio
from TikTokApi import TikTokApi

async def fetch_and_save_comments(video_id, output_file="comments.csv"):
    # Initialize TikTok API
    api = TikTokApi()

    try:
        # Fetch the video by its ID
        video = api.video(id=video_id)

        # Prepare a list to store comments
        comments_data = []

        # Fetch comments
        for comment in video.comments:
            comment_data = {
                "Comment ID": comment.id,
                "Author": comment.author.username,
                "Text": comment.text,
                "Likes": comment.likes_count
            }
            comments_data.append(comment_data)

            # Fetch replies for each comment (if any)
            async for reply in comment.replies():
                reply_data = {
                    "Comment ID": reply.id,
                    "Author": reply.author.username,
                    "Text": reply.text,
                    "Likes": reply.likes_count
                }
                comments_data.append(reply_data)

        # Convert the data to a DataFrame
        df = pd.DataFrame(comments_data)

        # Save to a CSV file
        df.to_csv(output_file, index=False)
        print(f"Comments have been saved to {output_file}")

    except Exception as e:
        print(f"An error occurred: {e}")

# Directly use await instead of asyncio.run()
video_id = "1234567890123456789"  # Replace with your TikTok video ID
output_file = "tiktok_comments.csv"

# Use await directly in the notebook environment
await fetch_and_save_comments(video_id, output_file)


An error occurred: 'method' object is not iterable


In [6]:
!pip install --upgrade TikTokApi
!pip install TikTokApi
!pip install pandas httpx


  Using cached TikTokApi-7.0.0-py3-none-any.whl.metadata (10 kB)
Using cached TikTokApi-7.0.0-py3-none-any.whl (55 kB)
  Attempting uninstall: TikTokApi
    Found existing installation: TikTokApi 4.0.0
    Uninstalling TikTokApi-4.0.0:
      Successfully uninstalled TikTokApi-4.0.0


In [25]:
import pandas as pd
from TikTokApi import TikTokApi

async def fetch_and_save_comments(video_id, output_file="comments.csv"):
    # Initialize TikTokApi
    api = TikTokApi()

    try:
        print("Fetching video...")
        # Fetch the video by its ID
        video = api.video(id=video_id)
        print(f"Video fetched: {video}")

        # Prepare a list to store comments
        comments_data = []

        # Fetch comments
        print("Fetching comments...")
        comments = await video.comments()  # Fetch comments for the video
        for comment in comments:
            print(f"Fetching comment: {comment.id}")
            comment_data = {
                "Comment ID": comment.id,
                "Author": comment.author.username,
                "Text": comment.text,
                "Likes": comment.likes_count
            }
            comments_data.append(comment_data)

            # Fetch replies for each comment (if any)
            replies = await comment.replies(count=20)  # Fetch replies for each comment
            for reply in replies:
                print(f"Fetching reply: {reply.id}")
                reply_data = {
                    "Comment ID": reply.id,
                    "Author": reply.author.username,
                    "Text": reply.text,
                    "Likes": reply.likes_count
                }
                comments_data.append(reply_data)

        # Convert the data to a DataFrame
        df = pd.DataFrame(comments_data)

        # Save to a CSV file
        df.to_csv(output_file, index=False)
        print(f"Comments have been saved to {output_file}")

    except Exception as e:
        print(f"An error occurred: {e}")

# Directly use await in a Jupyter notebook or asyncio.run in a script
video_id = "7415536407583755525"  # Replace with your TikTok video ID (only the ID part)
output_file = "tiktok_comments.csv"

# Use await in a Jupyter notebook or asyncio.run() in a script
await fetch_and_save_comments(video_id, output_file)


Fetching video...
Video fetched: TikTokApi.video(id='7415536407583755525')
Fetching comments...
An error occurred: object async_generator can't be used in 'await' expression


In [22]:
import asyncio
import requests
import httpx
from datetime import datetime
from typing import Optional, Union, AsyncIterator, Iterator
import json

# Helper functions
def extract_video_id_from_url(url: str, headers: dict, proxy: Optional[str] = None) -> str:
    """Extract video ID from the given URL."""
    # Extract the video ID (this is just an example, real extraction depends on the URL format)
    return url.split("/")[-1]

def requests_cookie_to_playwright_cookie(cookie):
    """Converts requests cookie format to Playwright format."""
    return {
        "name": cookie.name,
        "value": cookie.value,
        "domain": cookie.domain,
        "path": cookie.path,
        "expires": cookie.expires,
        "httpOnly": cookie.has_non_standard_attr('HttpOnly'),
        "secure": cookie.has_non_standard_attr('Secure')
    }

class InvalidResponseException(Exception):
    """Exception raised for invalid responses from TikTok."""
    pass

class Comment:
    def __init__(self, data):
        self.id = data.get("id")
        self.author = data.get("author", {}).get("username")
        self.text = data.get("text")
        self.likes_count = data.get("likes_count")

    def __repr__(self):
        return f"Comment(id={self.id}, author={self.author}, text={self.text})"

# Main TikTokApi class (simplified for this example)
class TikTokApi:
    def __init__(self):
        # Initialize session and other components as needed
        pass
    
    async def video(self, id: str):
        return Video(id=id, parent=self)

    async def make_request(self, url: str, params: dict, headers: dict, session_index: int):
        async with httpx.AsyncClient() as client:
            response = await client.get(url, params=params, headers=headers)
            return response.json()

class Video:
    parent: TikTokApi
    
    def __init__(self, id: Optional[str] = None, url: Optional[str] = None, data: Optional[dict] = None, **kwargs):
        self.id = id
        self.url = url
        self.as_dict = data
        self.parent = kwargs.get("parent")
        
    async def comments(self, count=20, cursor=0, **kwargs) -> Iterator[Comment]:
        found = 0
        while found < count:
            params = {
                "aweme_id": self.id,
                "count": 20,
                "cursor": cursor,
            }

            resp = await self.parent.make_request(
                url="https://www.tiktok.com/api/comment/list/",
                params=params,
                headers=kwargs.get("headers"),
                session_index=kwargs.get("session_index"),
            )

            if resp is None:
                raise InvalidResponseException(
                    resp, "TikTok returned an invalid response."
                )

            for video in resp.get("comments", []):
                yield Comment(video)
                found += 1

            if not resp.get("has_more", False):
                return

            cursor = resp.get("cursor")

    def __repr__(self):
        return f"TikTokApi.video(id='{self.id}')"

# The function to fetch comments and save to CSV
async def fetch_and_save_comments(video_id: str, output_file: str = "comments.csv"):
    api = TikTokApi()  # Initialize the TikTok API instance

    try:
        print("Fetching video...")
        video = await api.video(id=video_id)  # Fetch the video using its ID

        print("Fetching comments...")
        comments_data = []

        async for comment in video.comments(count=20):  # Fetch 20 comments asynchronously
            comment_data = {
                "Comment ID": comment.id,
                "Author": comment.author,
                "Text": comment.text,
                "Likes": comment.likes_count
            }
            comments_data.append(comment_data)

        # Save comments to CSV
        import pandas as pd
        df = pd.DataFrame(comments_data)
        df.to_csv(output_file, index=False)
        print(f"Comments have been saved to {output_file}")

    except Exception as e:
        print(f"An error occurred: {e}")

# Run the function
if __name__ == "__main__":
    video_id = "7415536407583755525"  # Replace with your TikTok video ID
    output_file = "tiktok_comments.csv"

    # Run the async function
    asyncio.run(fetch_and_save_comments(video_id, output_file))


RuntimeError: asyncio.run() cannot be called from a running event loop

In [30]:
import pandas as pd
from TikTokApi import TikTokApi

async def fetch_and_save_comments(video_id: str, output_file: str = "comments.csv"):
    api = TikTokApi()  # Initialize the TikTok API instance

    try:
        print("Fetching video...")
        video = api.video(id=video_id)  # No await here, as video fetching is not async

        print("Fetching comments...")
        comments_data = []

        async for comment in video.comments(count=20):  # Fetch comments asynchronously
            comment_data = {
                "Comment ID": comment.id,
                "Author": comment.author.username,
                "Text": comment.text,
                "Likes": comment.likes_count
            }
            comments_data.append(comment_data)

        # Save comments to CSV
        df = pd.DataFrame(comments_data)
        df.to_csv(output_file, index=False)
        print(f"Comments have been saved to {output_file}")

    except Exception as e:
        print(f"An error occurred: {e}")

# Use await to directly run the function in the notebook
video_id = "7415536407583755525"  # Replace with your TikTok video ID
output_file = "tiktok_comments.csv"

await fetch_and_save_comments(video_id, output_file)


Fetching video...
Fetching comments...
An error occurred: 'TikTokApi' object has no attribute 'num_sessions'


In [32]:
!pip install --upgrade TikTokApi

In [43]:
pip install TikTokApi pandas

Note: you may need to restart the kernel to use updated packages.


In [45]:
import pandas as pd
import asyncio
from TikTokApi import TikTokApi

async def fetch_and_save_comments(video_id, output_file="comments.csv"):
    # Initialize TikTok API
    api = TikTokApi.get_instance()  # Using the correct method to initialize TikTokApi

    try:
        print("Fetching video...")
        video = api.video(id=video_id)  # Fetch the video by ID
        print(f"Video fetched: {video}")

        print("Fetching comments...")
        comments_data = []  # List to store the comments

        # Fetch comments
        async for comment in video.comments():  # Async for loop to handle the comments
            comment_data = {
                "Comment ID": comment.id,
                "Author": comment.author.username,
                "Text": comment.text,
                "Likes": comment.likes_count
            }
            comments_data.append(comment_data)

            # Fetch replies for each comment (if any)
            async for reply in comment.replies(count=20):  # Async for loop for replies
                reply_data = {
                    "Comment ID": reply.id,
                    "Author": reply.author.username,
                    "Text": reply.text,
                    "Likes": reply.likes_count
                }
                comments_data.append(reply_data)

        # Convert the data into a pandas DataFrame
        df = pd.DataFrame(comments_data)

        # Save to a CSV file
        df.to_csv(output_file, index=False)
        print(f"Comments have been saved to {output_file}")

    except Exception as e:
        print(f"An error occurred: {e}")

# Provide the TikTok video ID
video_id = "7415536407583755525"  # Replace with your TikTok video ID (only the ID part)
output_file = "tiktok_comments.csv"

# Run the async function using asyncio
if __name__ == "__main__":
    asyncio.run(fetch_and_save_comments(video_id, output_file))


C:\Users\pc\AppData\Roaming\Python\Python312\site-packages\pygments\regexopt.py:26: RuntimeWarning: coroutine 'fetch_and_save_comments' was never awaited
  def regex_opt_inner(strings, open_paren):
C:\Users\pc\AppData\Roaming\Python\Python312\site-packages\pygments\regexopt.py:78: RuntimeWarning: coroutine 'fetch_and_save_comments' was never awaited
  for group in groupby(strings, lambda s: s[0] == first[0])) \


RuntimeError: asyncio.run() cannot be called from a running event loop

In [47]:
import pandas as pd
import asyncio
from TikTokApi import TikTokApi

async def fetch_and_save_comments(video_id, output_file="comments.csv"):
    # Initialize TikTok API
    api = TikTokApi.get_instance()  # Using the correct method to initialize TikTokApi

    try:
        print("Fetching video...")
        video = api.video(id=video_id)  # Fetch the video by ID
        print(f"Video fetched: {video}")

        print("Fetching comments...")
        comments_data = []  # List to store the comments

        # Fetch comments
        async for comment in video.comments():  # Async for loop to handle the comments
            comment_data = {
                "Comment ID": comment.id,
                "Author": comment.author.username,
                "Text": comment.text,
                "Likes": comment.likes_count
            }
            comments_data.append(comment_data)

            # Fetch replies for each comment (if any)
            async for reply in comment.replies(count=20):  # Async for loop for replies
                reply_data = {
                    "Comment ID": reply.id,
                    "Author": reply.author.username,
                    "Text": reply.text,
                    "Likes": reply.likes_count
                }
                comments_data.append(reply_data)

        # Convert the data into a pandas DataFrame
        df = pd.DataFrame(comments_data)

        # Save to a CSV file
        df.to_csv(output_file, index=False)
        print(f"Comments have been saved to {output_file}")

    except Exception as e:
        print(f"An error occurred: {e}")

# Provide the TikTok video ID
video_id = "7415536407583755525"  # Replace with your TikTok video ID (only the ID part)
output_file = "tiktok_comments.csv"

# In Jupyter or interactive environments, just await the function directly
await fetch_and_save_comments(video_id, output_file)


AttributeError: type object 'TikTokApi' has no attribute 'get_instance'

In [57]:
import pandas as pd
import asyncio
from TikTokApi import TikTokApi

async def fetch_and_save_comments(video_id, output_file="comments.csv"):
    # Initialize TikTok API correctly
    api = TikTokApi()  # Initialize the TikTokApi without using get_instance()

    try:
        print("Fetching video...")
        video = api.video(id=video_id)  # Fetch the video by ID
        print(f"Video fetched: {video}")

        print("Fetching comments...")
        comments_data = []  # List to store the comments

        # Fetch comments
        async for comment in video.comments():  # Async for loop to handle the comments
            comment_data = {
                "Comment ID": comment.id,
                "Author": comment.author.username,
                "Text": comment.text,
                "Likes": comment.likes_count
            }
            comments_data.append(comment_data)

            # Fetch replies for each comment (if any)
            async for reply in comment.replies(count=20):  # Async for loop for replies
                reply_data = {
                    "Comment ID": reply.id,
                    "Author": reply.author.username,
                    "Text": reply.text,
                    "Likes": reply.likes_count
                }
                comments_data.append(reply_data)

        # Convert the data into a pandas DataFrame
        df = pd.DataFrame(comments_data)

        # Save to a CSV file
        df.to_csv(output_file, index=False)
        print(f"Comments have been saved to {output_file}")

    except Exception as e:
        print(f"An error occurred: {e}")

# Provide the TikTok video ID
video_id = "7415536407583755525"  # Replace with your TikTok video ID (only the ID part)
output_file = "tiktok_comments.csv"

# In Jupyter or interactive environments, just await the function directly
await fetch_and_save_comments(video_id, output_file)


Fetching video...
Video fetched: TikTokApi.video(id='7415536407583755525')
Fetching comments...
An error occurred: 'TikTokApi' object has no attribute 'num_sessions'


In [69]:
import pandas as pd
from TikTokApi import TikTokApi

# Define the function to fetch comments
async def fetch_and_save_comments(video_id, output_file="comments.csv"):
    # Initialize TikTok API
    api = TikTokApi()

    try:
        # Fetch the video object
        print("Fetching video...")
        video = api.video(id=video_id)
        print(f"Video fetched: {video}")

        # Fetch the comments for the video
        print("Fetching comments...")
        comments = []
        async for comment in video.comments():
            comments.append({
                "user": comment.user.username,
                "comment": comment.text,
                "likes": comment.likes,
                "timestamp": comment.create_time
            })

        # Convert the data into a pandas DataFrame
        df = pd.DataFrame(comments)

        # Save to a CSV file
        df.to_csv(output_file, index=False)
        print(f"Comments have been saved to {output_file}")

    except Exception as e:
        print(f"An error occurred: {e}")

# In Jupyter or interactive environments, just await the function directly
video_id = '7415536407583755525'  # Replace with the actual video ID
output_file = "tiktok_comments.csv"

# Run the fetch_and_save_comments function
await fetch_and_save_comments(video_id, output_file)


Fetching video...
Video fetched: TikTokApi.video(id='7415536407583755525')
Fetching comments...
An error occurred: 'TikTokApi' object has no attribute 'num_sessions'


In [27]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Initialize the WebDriver with a user-agent
options = webdriver.ChromeOptions()
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
driver = webdriver.Chrome(options=options)

try:
    # Open the TikTok URL
    driver.get("https://www.tiktok.com/@the.ambara.fashio/video/7465674803433819398")

    # Wait for the page to load
    time.sleep(5)

    # Accept the prompt (if it appears)
    try:
        accept_button = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Accept')]"))
        )
        accept_button.click()
        print("Prompt accepted.")
    except Exception as e:
        print("Error accepting prompt:", e)

    # Scroll and wait for comments to load
    for _ in range(5):  # Adjust the number of scrolls as needed
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        try:
            WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.comment-item'))
            )
        except:
            print("No new comments loaded after scroll.")
        time.sleep(3)  # Increase sleep time if necessary

    # Locate comments
    comments = driver.find_elements(By.CSS_SELECTOR, '.comment-item')  # Replace with the correct selector

    # Save comments to a file
    if comments:
        with open("tiktok_comments.txt", "w", encoding="utf-8") as file:
            for comment in comments:
                file.write(comment.text + "\n")
        print(f"{len(comments)} comments have been saved to 'tiktok_comments.txt'.")
    else:
        print("No comments found.")

finally:
    # Close the WebDriver
    driver.quit()

Error accepting prompt: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6F3FF02F5+28725]
	(No symbol) [0x00007FF6F3F52AE0]
	(No symbol) [0x00007FF6F3DE510A]
	(No symbol) [0x00007FF6F3E393D2]
	(No symbol) [0x00007FF6F3E395FC]
	(No symbol) [0x00007FF6F3E83407]
	(No symbol) [0x00007FF6F3E5FFEF]
	(No symbol) [0x00007FF6F3E80181]
	(No symbol) [0x00007FF6F3E5FD53]
	(No symbol) [0x00007FF6F3E2A0E3]
	(No symbol) [0x00007FF6F3E2B471]
	GetHandleVerifier [0x00007FF6F431F30D+3366989]
	GetHandleVerifier [0x00007FF6F43312F0+3440688]
	GetHandleVerifier [0x00007FF6F43278FD+3401277]
	GetHandleVerifier [0x00007FF6F40BAAAB+858091]
	(No symbol) [0x00007FF6F3F5E74F]
	(No symbol) [0x00007FF6F3F5A304]
	(No symbol) [0x00007FF6F3F5A49D]
	(No symbol) [0x00007FF6F3F48B69]
	BaseThreadInitThunk [0x00007FF89EF0E8D7+23]
	RtlUserThreadStart [0x00007FF89F37BF2C+44]

No new comments loaded after scroll.
No new comments loaded after scroll.
No new comments loaded after scroll.
No new comments loaded after scroll.
No 

In [8]:
pip install TikTokApi playwright


In [3]:
import requests
from bs4 import BeautifulSoup

# Function to scrape TikTok page
def scrape_tiktok(username):
    url = f"https://www.tiktok.com/@{username}"
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})  # Add a User-Agent header

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # Example: find all video links
        video_links = []
        for link in soup.find_all('a', href=True):
            if '/video/' in link['href']:  # Filtering for video links
                video_links.append(f"https://www.tiktok.com{link['href']}")

        return set(video_links)
    else:
        print(f"Failed to retrieve data for {username} (Status Code: {response.status_code})")
        return []

# Usage
username = 'anuu__21'  # Replace with the actual TikTok username
videos = scrape_tiktok(username)

for video in videos:
    print(video)


In [3]:
pip install selenium webdriver-manager


Note: you may need to restart the kernel to use updated packages.


In [3]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.microsoft import EdgeChromiumDriverManager
import time

# Setup Edge options
edge_options = Options()
edge_options.add_argument("--disable-blink-features=AutomationControlled")  # Bypass bot detection
edge_options.add_argument("--start-maximized")
edge_options.add_argument("--headless")  # Run in headless mode (optional, remove if you want UI)
edge_options.add_argument("--disable-gpu")
edge_options.add_argument("--no-sandbox")
edge_options.add_argument("--disable-dev-shm-usage")

# Initialize Edge WebDriver
driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()), options=edge_options)

# Replace with your video URL
VIDEO_URL = "https://www.tiktok.com/@anuu__21/video/7415536407583755525"  # Change this to your actual video link

try:
    driver.get(VIDEO_URL)
    time.sleep(5)  # Wait for the page to load

    # Scroll down to load more comments
    for _ in range(5):  # Adjust the range for more comments
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(3)

    # Extract comments (change the selector based on the website)
    comments = driver.find_elements(By.XPATH, "//div[contains(@class, 'comment')]")  # Update selector if needed

    # Print extracted comments
    print("\n✅ Downloaded Comments:")
    for idx, comment in enumerate(comments, 1):
        print(f"{idx}. {comment.text}")

    print(f"\nTotal Comments Extracted: {len(comments)}")

except Exception as e:
    print("❌ Error Occurred:", str(e))

finally:
    driver.quit()  # Close the browser



✅ Downloaded Comments:

Total Comments Extracted: 0


In [25]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.microsoft import EdgeChromiumDriverManager
import time

# Set up options for Edge browser
edge_options = Options()
edge_options.add_argument("--disable-blink-features=AutomationControlled")  # Avoid bot detection
edge_options.add_argument("--start-maximized")
edge_options.add_argument("--headless")  # Remove this if you want to see the browser
edge_options.add_argument("--disable-gpu")
edge_options.add_argument("--no-sandbox")
edge_options.add_argument("--disable-dev-shm-usage")

# Start Edge WebDriver
driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()), options=edge_options)

# Change this to your video URL
VIDEO_URL = "https://www.tiktok.com/@anuu__21/video/7415536407583755525"  # Update this to the actual video link

try:
    driver.get(VIDEO_URL)
    time.sleep(5)  # Wait for the page to load

    # Scroll down several times to load more comments
    total_scrolls = 7  # Number of times to scroll down
    extracted_comments = []

    for _ in range(total_scrolls):
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(2)

        # Update the XPath to find the TikTok comment section
        comments_xpath = "//div[@class='tiktok-1sxi6g2-DivCommentItemContainer']"  # Adjust this for TikTok
        comments = driver.find_elements(By.XPATH, comments_xpath)

        # Check how many comments we found
        if comments:
            extracted_comments = comments
            print(f"\n✅ Extracted {len(extracted_comments)} comments so far.")

        # Stop scrolling if we have at least 20% of the expected total comments
        if len(extracted_comments) >= 0.2 * 100:  # Adjust 100 to your estimated number of comments
            break

    if extracted_comments:
        print("\n✅ Downloaded Comments:")
        for idx, comment in enumerate(extracted_comments, 1):
            print(f"{idx}. {comment.text}")
    else:
        print("\n⚠️ No comments found! Try updating the XPath.")

    print(f"\nTotal Comments Extracted: {len(extracted_comments)}")

except Exception as e:
    print("❌ Error Occurred:", str(e))

finally:
    driver.quit()  # Close the browser


⚠️ No comments found! Try updating the XPath.

Total Comments Extracted: 0
